In [0]:
%sql
-- Cria o banco de dados (se ainda não existir)
CREATE DATABASE IF NOT EXISTS kyc_db
LOCATION 'dbfs:/mnt/datalake/silver/kyc_risk_analysis';

-- Registra a tabela Silver
CREATE TABLE IF NOT EXISTS kyc_db.client_transactions_risk
USING DELTA
LOCATION 'dbfs:/mnt/datalake/silver/kyc_risk_analysis/client_transactions_risk';

-- Registra a tabela Gold
CREATE TABLE IF NOT EXISTS kyc_db.aggregated_client_risk
USING DELTA
LOCATION 'dbfs:/mnt/datalake/gold/kyc_risk_analysis/aggregated_client_risk';
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from datetime import datetime, timedelta
import random



In [0]:

def save_csv(df, sub_path):
    (
        df.coalesce(1)
        .write
        .mode("overwrite")
        .option("header", True)
        .csv(f"{output_path}/{sub_path}")
    )